### Pairwise Evaluation

    두 개 이상의 LLM 생성물 서로 비교한다.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def evaluate_pairwise(runs: list, example) -> dict:

    # 점수 저장
    scores = {}
    for i, run in enumerate(runs):
        scores[run.id] = i
    

    # 각 예제에 대한 실행 쌍
    answer_a = runs[0].outputs["answer"]
    answer_b = runs[1].outputs["answer"]
    question = example.inputs["question"]

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    grade_prompt = PromptTemplate.from_template(
        """
        You are an LLM judge. Compare the following two answers to a question and determine which one is better.
        Better answer is the one that is more detailed and informative.
        If the answer is not related to the question, it is not a good answer.

        
        # Question:
        {question}
        
        #Answer A: 
        {answer_a}
        
        #Answer B: 
        {answer_b}
        
        Output should be either `A` or `B`. Pick the answer that is better.
        
        #Preference:
        """
    )
    answer_grader = grade_prompt | llm | StrOutputParser()

    score = answer_grader.invoke(
        {
            "question": question,
            "answer_a": answer_a,
            "answer_b": answer_b
        }
    )

    if score == "A": # A가 더 답변을 잘했다.
        scores[runs[0].id] = 1
        scores[runs[1].id] = 0
    elif score == "B": # B가 더 답변을 잘했다.
        scores[runs[0].id] = 0
        scores[runs[1].id] = 1
    else:
        scores[runs[0].id] = 0
        scores[runs[1].id] = 0
        
    return {"key": "ranked_preference", "scores": scores}

In [19]:
from rag import PDFRAG
from langchain_openai import ChatOpenAI

def ask_question_with_llm(llm):
    
    rag = PDFRAG(
        "data/snow-white.pdf",
        llm
    )
    
    retriever = rag.create_retriever()
    
    rag_chain = rag.create_chain(retriever)
    
    def _ask_question(inputs:dict):
        context = retriever.invoke(inputs["question"])
        context = "\n".join([doc.page_content for doc in context])
        return {
            "question" : inputs["question"],
            "context" : context,
            "answer" : rag_chain.invoke(inputs["question"])
        }
        
    return _ask_question

In [12]:
from langchain_openai import ChatOpenAI

gpt3 = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

gpt3.invoke("안녕하세요?")

AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 13, 'total_tokens': 34, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-85bbef3f-4704-4cdd-bdd1-a87a467547e7-0', usage_metadata={'input_tokens': 13, 'output_tokens': 21, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

Ollama 사용 시

In [14]:
from langchain_ollama import ChatOllama

# Ollama 모델을 불러옵니다.
ollama = ChatOllama(model="")

# Ollama 모델 호출
ollama.invoke("안녕하세요?")

ModuleNotFoundError: No module named 'langchain_ollama'

In [20]:
gpt4o_chain = ask_question_with_llm(ChatOpenAI(model="gpt-4o-mini", temperature=0))
gpt3_chain = ask_question_with_llm(ChatOpenAI(model="gpt-3.5-turbo", temperature=0))

# ollama 사용 시
# ollama_chai = ask_question_with_ll(ChatOllama(model=""))

In [16]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

cot_qa_evaluator = LangChainStringEvaluator(
    "cot_qa",
    config={"llm": ChatOpenAI(model="gpt-4o-mini", temperature=0)}, # 평가자
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": run.outputs["context"],
        "input": example.inputs["question"]
    }
)

dataset_name = "RAG_EVALUATION_DATASET"

experiment_result1 = evaluate(
    gpt3_chain,
    data=dataset_name,
    evaluators=[cot_qa_evaluator],
    experiment_prefix="MODEL_COMPARE_EVALUATION",
    metadata={
        "variant": "GPT-3.5-turbo 평가 (cot_qa)"
    }
)

experiment_result2 = evaluate(
    gpt4o_chain,
    data=dataset_name,
    evaluators=[cot_qa_evaluator],
    experiment_prefix="MODEL_COMPARE_EVALUATION",
    metadata={
        "variant": "GPT-4o-mini 평가 (cot_qa)"
    }
)

View the evaluation results for experiment: 'MODEL_COMPARE_EVALUATION-51cc2999' at:
https://smith.langchain.com/o/5924f60f-3d1b-477e-8828-acaa0f00dd1c/datasets/cb8ba601-ec75-4365-a363-d0bd84615bc1/compare?selectedSessions=64f2d8a9-a681-46b2-9255-4b9f80c784d6




0it [00:00, ?it/s]

View the evaluation results for experiment: 'MODEL_COMPARE_EVALUATION-5a34e825' at:
https://smith.langchain.com/o/5924f60f-3d1b-477e-8828-acaa0f00dd1c/datasets/cb8ba601-ec75-4365-a363-d0bd84615bc1/compare?selectedSessions=e0198eb7-75ab-411c-9e57-2ea40747414e




0it [00:00, ?it/s]

In [21]:
from langsmith.evaluation import evaluate_comparative

evaluate_comparative(
    {"MODEL_COMPARE_EVALUATION-5a34e825", "MODEL_COMPARE_EVALUATION-51cc2999"},
    
    # 평가자
    evaluators=[evaluate_pairwise]
)

View the pairwise evaluation results at:
https://smith.langchain.com/o/5924f60f-3d1b-477e-8828-acaa0f00dd1c/datasets/cb8ba601-ec75-4365-a363-d0bd84615bc1/compare?selectedSessions=64f2d8a9-a681-46b2-9255-4b9f80c784d6%2Ce0198eb7-75ab-411c-9e57-2ea40747414e&comparativeExperiment=bc5119c6-a0af-4774-a112-9a08f05f3a22




  0%|          | 0/5 [00:00<?, ?it/s]